In [1]:
sc.setCheckpointDir('checkpoint/')
import os
import json
import numpy as np
import pandas as pd
import shutil
from pyspark.ml.recommendation import ALS
from pyspark.ml.recommendation import ALSModel
from pyspark.sql.functions import lit
from pyspark.sql.functions import lit, col
from pyspark.sql.types import IntegerType

from multiprocessing import Pool


TRAIN_FILE = 'data/full-interactions.csv'
USER_COL = "playlist"
ITEM_COL = "track"

MAPPED_CHALLENGE_FILE = "data/challenge_set_mapped.json"
REVERSED_TRACK_MAP_FILE = 'data/mpd/reduced/reversed_track_map.json'

MAP_FILE = 'data/mpd/reduced/maps.json'

def load_ds():
    """ Carga TRAIN_FILE a un dataframe de spark. """
    ds = spark.read.csv(TRAIN_FILE, header=True, mode="DROPMALFORMED")
    ds = ds.withColumn(ITEM_COL, ds[ITEM_COL].cast(IntegerType()))
    ds = ds.withColumn(USER_COL, ds[USER_COL].cast(IntegerType()))
    ds = ds.withColumn("rating", lit(1))
    ds = ds.na.fill(-1).cache()
    return ds

def create_and_save_als(n_factors=30):
    """ Ejecuta ALS con @n_factors sobre TRAIN_FILE y lo graba en models/<<FACTORES>-factors-als/. """
    print("Creando...")
    ds = load_ds()
    als = ALS(rank=n_factors, userCol=USER_COL, itemCol=ITEM_COL, ratingCol="rating", implicitPrefs=True).fit(ds)
    als.save("models/{}-factors-als".format(n_factors))

    
def predict_playlists(n_factors=30):
    """ Genera spark/output/<FACTORS>-factors-als-playlists.json. 
            diccionario
                <tracks unicas de las playlists del challenge> ==> <playlists recomendadas ordenadas y con peso>
    
    """
    print("Prediciendo...")
    als = ALSModel.load("models/{}-factors-als".format(n_factors))    
    eval_playlists = json.load(open(MAPPED_CHALLENGE_FILE))
    
    # Obtengo tracks de la playlists
    tracks = list(set([track for playlist in eval_playlists for track in playlist['tracks']]))
    print("Total de tracks en el challenge set: {}".format(len(tracks)))
    tracks_ds = spark.createDataFrame(pd.DataFrame(tracks, columns=[ITEM_COL]))
    tracks_ds = tracks_ds.withColumn(ITEM_COL, tracks_ds[ITEM_COL].cast(IntegerType()))
    
    # Obtengo playlists de los tracks, 20 por track
    recommended_playlists = als.recommendForItemSubset(tracks_ds, 20)
    result = recommended_playlists.collect()
    
    json.dump(result, open("spark-output/{}-factors-als-playlists.json".format(n_factors), "w"))

def remove_duplicates(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]
    
def process_chunk_new(eval_playlists):
    res = {}
    for i, playlist in enumerate(eval_playlists):
        print("Playlist {}: {}".format(i, playlist['pid']))
        tracks = playlist['tracks']
        
        if not tracks:
            continue
        
        # playlists recomendadas para cada track
        playlists_with_weights = pd.DataFrame([p for track in tracks for p in data[track]], 
                                              columns=["playlist", "weight"])
          
        tracks_df = dataset.merge(playlists_with_weights, on='playlist').groupby("track").agg({"weight": "sum"})
        lst = tracks_df.sort_values("weight", ascending=False).head(2000).index.tolist()
        
        for track in tracks:
            try:
                lst.remove(track)
            except:
                pass
        
        lst = remove_duplicates(lst)
        
        if len(lst) < 500:
            print("CUIDADO")
            print("FALLO LA CANTIDAD PARA LA PLAYLIST {}, DIO {}".format(playlist['pid'], len(lst)))
            print("CUIDADO")
            
        res[playlist['pid']] = lst[:500]
        
        if i > 0 and i % 50 == 0:
            print("Dumping...")
            json.dump(res, open("spark-output/results-{}/{}.json".format(n_factors, playlist['pid']), "w"))
            res = {}

    if res:
        print("Dumping...")
        json.dump(res, open("spark-output/results-{}/{}.json".format(n_factors, playlist['pid']), "w"))

def get_most_popular(dataset, rev_map):
    most_popular = dataset.groupby("track").size().sort_values(ascending=False).head(500).index.tolist()
    most_popular_mapped = ["spotify:track:{}".format(rev_map[str(t)]) for t in most_popular]
    return most_popular_mapped

def get_premapped_recommendations(n_factors=30):
    pre_mapped_recommendations = {}
    files = [json.load(open("spark-output/results-{}/{}".format(n_factors,f))) for f in os.listdir("spark-output/results-{}/".format(n_factors))
              if f.endswith(".json")]
    for f in files:
        pre_mapped_recommendations.update(f)
    return pre_mapped_recommendations

def add_header(f):
    header = "team_info,main,Palmas Social Club,julianpeller@gmail.com"
    with open(f, 'r') as original: data = original.read()
    with open(f, 'w') as modified: modified.write("{}\n".format(header) + data)    
        
def take_n(here, but_not_here, n):
    res = []
    for e in here:
        if e not in but_not_here:
            res.append(e)
            if len(res) == n:
                return res
    print("ERROR in take_n")

def final_step(n_factors):
    print("Entre al paso final")
    pre_mapped_recommendations = get_premapped_recommendations(n_factors)
    result_lines = []
    for i, pl in enumerate(eval_playlists):
        try:
            r = [pl['pid']] + ["spotify:track:{}".format(rev_map[str(t)]) 
                                   for t in pre_mapped_recommendations[str(pl['pid'])]]

            if len(r) < 501:
                print("filling!!!")
                fill = 501 - len(r)
                existing = ["spotify:track:{}".format(rev_map[str(t)]) for t in pl['tracks']]
                r = r + take_n(most_popular, r[1:] + existing, fill)

        except Exception as e:
            r = [pl['pid']] + most_popular
            if i > 1000:
                print("Failed {}, using most popular".format(pl['pid']))
        result_lines.append(r)
    
    df = pd.DataFrame(result_lines)
    df.to_csv("spark-output/deliverable-{}.csv".format(n_factors), header=False, index=False)
    add_header("spark-output/deliverable-{}.csv".format(n_factors))
    print("Termine")

In [2]:
# Todas tienen que ser globales
num_threads = 32

dataset = pd.read_csv(TRAIN_FILE) #, nrows=200000)
eval_playlists = json.load(open(MAPPED_CHALLENGE_FILE))#[1000:1200]

chunks = np.array_split(eval_playlists, num_threads)

rev_map = json.load(open(REVERSED_TRACK_MAP_FILE))
most_popular = get_most_popular(dataset, rev_map)

In [3]:
# %%time

# n_factors=4

# try:
#     os.makedirs("spark-output/results-{}".format(n_factors))
# except:
#     pass
# try:
#     shutil.rmtree("models/{}-factors-als".format(n_factors))
# except:
#     pass

# # Hace ALS y lo graba en models/
# create_and_save_als(n_factors)
# # Carga ALS y escribe archivo "spark-output/{}-factors-als-playlists.json
# predict_playlists(n_factors)

# # Carga archivo spark-output/{}-factors-als-playlists.json. Data tiene que existir global
# # Genera archivos spark-output/results-{}/* con 500 playlists para cada playlist, en ids viejos
# data = dict(json.load(open("spark-output/{}-factors-als-playlists.json".format(n_factors))))

# pool = Pool(num_threads)
# pool.map(process_chunk_new, chunks)
# pool.close()

# # Carga spark-output/results-{}/*, lo mappea a uris de spotify, rellena con most popular los que no tienen
# final_step(n_factors)

# pd.read_csv("spark-output/deliverable-{}.csv".format(n_factors), skiprows=1,header=None).head()

In [4]:
%%time
for n_factors in [40, 60, 80]:
        try:
            os.makedirs("spark-output/results-{}".format(n_factors))
        except:
            pass
        try:
            shutil.rmtree("models/{}-factors-als".format(n_factors))
        except:
            pass
        create_and_save_als(n_factors)
        predict_playlists(n_factors)
        print("De playlists a tracks...")
        data = dict(json.load(open("spark-output/{}-factors-als-playlists.json".format(n_factors))))
        pool = Pool(num_threads)
        pool.map(process_chunk_new, chunks)
        pool.close()
        print("Dumpeando...")
        final_step(n_factors)

Creando...
Prediciendo...
Total de tracks en el challenge set: 66243
De playlists a tracks...
Playlist 0: 1000002
Playlist 0: 1000856
Playlist 0: 1001642
Playlist 1: 1000858
Playlist 1: 1000003
Playlist 1: 1001643
Playlist 2: 1000860
Playlist 2: 1000004
Playlist 0: 1002486
Playlist 0: 1001388
Playlist 2: 1001644
Playlist 3: 1000006
Playlist 1: 1002491
Playlist 3: 1001648
Playlist 4: 1000007
Playlist 0: 1002743
Playlist 3: 1000861
Playlist 2: 1002493
Playlist 4: 1000865
Playlist 4: 1001649
Playlist 5: 1000868
Playlist 3: 1002495
Playlist 0: 1007627
Playlist 6: 1000869
Playlist 0: 1008740
Playlist 4: 1002497
Playlist 0: 1010988
Playlist 5: 1000010
Playlist 7: 1000871
Playlist 0: 1012084
Playlist 0: 1009894
Playlist 5: 1001652
Playlist 8: 1000873
Playlist 6: 1000011
Playlist 6: 1001659
Playlist 9: 1000880
Playlist 7: 1000013
Playlist 0: 1013170
Playlist 7: 1001660
Playlist 5: 1002501
Playlist 10: 1000882
Playlist 8: 1001664
Playlist 6: 1002502
Playlist 8: 1000014
Playlist 9: 1001665
Playl

Playlist 97: 1001882
Playlist 100: 1000263
Playlist 102: 1001111
Playlist 98: 1001885
Playlist 103: 1001112
Playlist 99: 1001888
Playlist 104: 1001115
Playlist 100: 1001889
Playlist 105: 1001119
Playlist 101: 1001896
Playlist 102: 1001899
Playlist 106: 1001121
Playlist 103: 1001907
Playlist 107: 1001123
Playlist 108: 1001124
Playlist 104: 1001908
Playlist 105: 1001910
Playlist 109: 1001130
Playlist 101: 1000267
Playlist 110: 1001134
Playlist 111: 1001135
Playlist 102: 1000268
Playlist 106: 1001916
Playlist 103: 1000271
Playlist 107: 1001917
Playlist 108: 1001919
Playlist 104: 1000272
Playlist 105: 1000275
Playlist 112: 1001136
Playlist 109: 1001920
Playlist 113: 1001137
Playlist 106: 1000278
Playlist 114: 1001142
Playlist 107: 1000279
Playlist 110: 1001925
Playlist 115: 1001146
Playlist 111: 1001928
Playlist 116: 1001147
Playlist 117: 1001148
Playlist 108: 1000281
Playlist 118: 1001149
Playlist 112: 1001929
Playlist 119: 1001151
Playlist 113: 1001931
Playlist 114: 1001935
Playlist 120:

Playlist 230: 1002254
Playlist 227: 1001428
Playlist 231: 1002256
Playlist 215: 1000569
Playlist 228: 1001429
Playlist 229: 1001430
Playlist 216: 1000570
Playlist 217: 1000572
Playlist 230: 1001434
Playlist 232: 1002258
Playlist 231: 1001439
Playlist 233: 1002259
Playlist 234: 1002263
Playlist 218: 1000573
Playlist 232: 1001440
Playlist 235: 1002267
Playlist 219: 1000583
Playlist 233: 1001444
Playlist 236: 1002269
Playlist 234: 1001447
Playlist 235: 1001451
Playlist 237: 1002270
Playlist 220: 1000584
Playlist 236: 1001453
Playlist 221: 1000585
Playlist 238: 1002274
Playlist 222: 1000589
Playlist 239: 1002275
Playlist 240: 1002276
Playlist 223: 1000591
Playlist 224: 1000596
Playlist 241: 1002279
Playlist 237: 1001455
Playlist 225: 1000604
Playlist 242: 1002280
Playlist 243: 1002282
Playlist 238: 1001458
Playlist 244: 1002283
Playlist 226: 1000616
Playlist 239: 1001460
Playlist 240: 1001461
Playlist 227: 1000622
Playlist 241: 1001463
Playlist 228: 1000626
Playlist 242: 1001464
Playlist 2

Playlist 5: 1004247
Playlist 5: 1004736
Playlist 5: 1000098
Playlist 5: 1009154
Playlist 5: 1004437
Playlist 5: 1045253
Playlist 5: 1006525
Playlist 5: 1009903
Playlist 5: 1034278
Playlist 5: 1011003
Playlist 5: 1013192
Playlist 5: 1012106
Playlist 5: 1008758
Playlist 5: 1028816
Playlist 5: 1007649
Playlist 5: 1013545
Playlist 5: 1005423
Playlist 5: 1001412
Playlist 5: 1018078
Playlist 5: 1023743
Playlist 5: 1039724
Playlist 66: 1000049
Playlist 5: 1002755
Playlist 5: 1006797
Playlist 5: 1002415
Playlist 5: 1005863
Playlist 5: 1003661
Playlist 5: 1003294
Playlist 5: 1011306
Playlist 6: 1004249
Playlist 6: 1004739
Playlist 6: 1000103
Playlist 6: 1011011
Playlist 6: 1009155
Playlist 6: 1045274
Playlist 6: 1004444
Playlist 6: 1006526
Playlist 6: 1009915
Playlist 6: 1034287
Playlist 6: 1013196
Playlist 6: 1012109
Playlist 6: 1008761
Playlist 6: 1013557
Playlist 6: 1028851
Playlist 6: 1007653
Playlist 6: 1005426
Playlist 6: 1001413
Playlist 6: 1018105
Playlist 67: 1000052
Playlist 6: 100275

Playlist 18: 1001495
Playlist 18: 1002520
Playlist 18: 1039909
Playlist 18: 1029066
Playlist 79: 1000115
Playlist 18: 1003711
Playlist 19: 1004282
Playlist 18: 1006903
Playlist 18: 1003321
Playlist 18: 1005903
Playlist 19: 1000220
Playlist 19: 1013234
Playlist 19: 1007684
Playlist 19: 1004772
Playlist 19: 1012172
Playlist 19: 1018308
Playlist 19: 1008811
Playlist 19: 1045424
Playlist 19: 1013621
Playlist 19: 1034623
Playlist 19: 1024027
Playlist 19: 1002778
Playlist 19: 1009974
Playlist 19: 1005469
Playlist 19: 1009284
Playlist 19: 1011426
Playlist 19: 1006553
Playlist 19: 1011068
Playlist 19: 1001496
Playlist 19: 1004552
Playlist 19: 1039917
Playlist 19: 1002521
Playlist 20: 1004283
Playlist 19: 1029082
Playlist 19: 1003719
Playlist 80: 1000116
Playlist 19: 1006915
Playlist 19: 1003322
Playlist 19: 1005906
Playlist 20: 1000221
Playlist 20: 1007685
Playlist 20: 1012178
Playlist 20: 1013238
Playlist 20: 1004773
Playlist 20: 1008818
Playlist 20: 1018320
Playlist 20: 1013627
Playlist 20: 

...

Playlist 252: 1006249
Dumping...
Playlist 251: 1004006
Dumping...
Playlist 251: 1013163
Playlist 252: 1014037
Playlist 252: 1006277
Playlist 250: 1016600
Playlist 252: 1007406
Dumping...
Playlist 252: 1002635
Playlist 252: 1006594
Playlist 252: 1004084
Dumping...
Playlist 251: 1033038
Playlist 249: 1002639
Playlist 250: 1043615
Playlist 252: 1008489
Playlist 250: 1027958
Dumping...
Playlist 251: 1001893
Playlist 252: 1011905
Playlist 252: 1012940
Playlist 253: 1004477
Playlist 253: 1005184
Dumping...
Playlist 251: 1008624
Playlist 252: 1010796
Playlist 253: 1005656
Playlist 252: 1003182
Playlist 252: 1010848
Dumping...
Playlist 251: 1022542
Playlist 253: 1009688
Playlist 252: 1013171
Dumping...
Playlist 251: 1038400
Playlist 253: 1006261
Playlist 252: 1004008
Playlist 253: 1014041
Playlist 253: 1006278
Dumping...
Playlist 251: 1016675
Playlist 253: 1007429
Playlist 253: 1002636
Playlist 253: 1006598
Playlist 250: 1002647
Playlist 253: 1004086
Playlist 252: 1033039
Dumping...
Playlist 2

Playlist 265: 1002661
Playlist 265: 1006614
Playlist 262: 1028091
Playlist 264: 1003198
Playlist 265: 1007480
Playlist 263: 1016941
Playlist 265: 1014080
Playlist 264: 1033274
Playlist 263: 1038652
Playlist 264: 1013232
Playlist 264: 1004048
Playlist 266: 1009742
Playlist 265: 1010936
Playlist 266: 1006313
Playlist 262: 1003317
Playlist 266: 1005695
Playlist 266: 1005231
Playlist 265: 1008530
Playlist 264: 1008755
Playlist 264: 1002012
Playlist 265: 1010836
Playlist 266: 1004534
Playlist 266: 1004169
Playlist 266: 1006351
Playlist 263: 1043939
Playlist 264: 1022771
Playlist 265: 1011936
Playlist 265: 1012980
Playlist 266: 1006615
Playlist 266: 1002662
Playlist 265: 1003202
Playlist 263: 1028115
Playlist 266: 1007481
Playlist 265: 1033305
Playlist 264: 1016942
Playlist 265: 1004057
Playlist 266: 1014082
Playlist 264: 1038665
Playlist 265: 1013242
Playlist 267: 1009743
Playlist 265: 1002017
Playlist 266: 1010947
Playlist 267: 1006320
Playlist 266: 1008531
Playlist 267: 1005701
Playlist 2

Playlist 278: 1014107
Playlist 277: 1013297
Playlist 275: 1044340
Playlist 278: 1002682
Playlist 277: 1004097
Playlist 277: 1010877
Playlist 277: 1011978
Playlist 279: 1009797
Playlist 276: 1017237
Playlist 279: 1007508
Playlist 279: 1005738
Playlist 278: 1008590
Playlist 278: 1011035
Playlist 276: 1038997
Playlist 277: 1033554
Playlist 277: 1022998
Playlist 278: 1013037
Playlist 279: 1004572
Playlist 280: 1004223
Playlist 279: 1006379
Playlist 279: 1005298
Playlist 279: 1006660
Playlist 278: 1003230
Playlist 277: 1008869
Playlist 276: 1028261
Playlist 279: 1002683
Playlist 275: 1003515
Playlist 279: 1006475
Playlist 278: 1002151
Playlist 278: 1013332
Playlist 278: 1010881
Playlist 276: 1044342
Playlist 279: 1014114
Playlist 280: 1009801
Playlist 278: 1004102
Playlist 280: 1007509
Playlist 278: 1011979
Playlist 277: 1017238
Playlist 280: 1005742
Playlist 279: 1008593
Playlist 281: 1004230
Playlist 279: 1011038
Playlist 280: 1004573
Playlist 277: 1039006
Playlist 278: 1023034
Playlist 2

Playlist 291: 1006570
Playlist 290: 1012004
Playlist 291: 1002224
Playlist 292: 1005791
Playlist 292: 1007559
Playlist 293: 1004309
Playlist 288: 1044584
Playlist 290: 1023235
Playlist 292: 1004615
Playlist 291: 1008651
Playlist 290: 1033726
Playlist 293: 1009841
Playlist 291: 1011102
Playlist 291: 1013075
Playlist 292: 1002706
Playlist 291: 1013413
Playlist 290: 1017430
Playlist 292: 1014145
Playlist 291: 1004138
Playlist 292: 1005351
Playlist 288: 1003545
Playlist 292: 1006436
Playlist 290: 1008931
Playlist 292: 1006703
Playlist 289: 1028415
Playlist 291: 1003249
Playlist 289: 1039283
Playlist 291: 1010923
Playlist 292: 1006571
Playlist 291: 1012006
Playlist 294: 1004312
Playlist 293: 1007563
Playlist 292: 1002225
Playlist 293: 1005794
Playlist 289: 1044605
Playlist 291: 1023239
Playlist 293: 1004616
Playlist 292: 1008657
Playlist 291: 1033751
Playlist 294: 1009843
Playlist 292: 1013085
Playlist 293: 1002710
Playlist 292: 1011103
Playlist 293: 1006439
Playlist 291: 1017469
Playlist 2

Playlist 303: 1003266
Playlist 303: 1013494
Playlist 304: 1006734
Dumping...
Playlist 301: 1039478
Playlist 300: 1003600
Playlist 302: 1009065
Dumping...
Playlist 304: 1006487
Playlist 301: 1028586
Playlist 303: 1004199
Playlist 304: 1002726
Playlist 305: 1005829
Playlist 304: 1005379
Playlist 305: 1004666
Playlist 304: 1008706
Playlist 304: 1013130
Playlist 303: 1023570
Playlist 305: 1007604
Playlist 304: 1011183
Dumping...
Playlist 301: 1044884
Playlist 305: 1000001
Playlist 304: 1012045
Playlist 304: 1002332
Playlist 306: 1009879
Playlist 303: 1017818
Playlist 303: 1033992
Playlist 305: 1006751
Playlist 306: 1004390
Playlist 304: 1010965
Playlist 304: 1003268
Playlist 304: 1013504
Playlist 305: 1006745
Playlist 302: 1039492
Playlist 305: 1002728
Dumping...
Playlist 301: 1003606
Playlist 305: 1006488
Playlist 303: 1009066
Playlist 302: 1028638
Playlist 304: 1004201
Playlist 305: 1005380
Playlist 306: 1005832
Playlist 306: 1004673
Playlist 305: 1013134
Playlist 305: 1008714
Playlist 3